# Risk Adjustment Ratio Index Builder
## Compute R Sharpe and Sortino-Ratios to compute a RAR-weighted-Portfolio for S&P stocks
### by Konstantin Smirnov


Following libraries are necessary

In [1]:
import pandas as pd
import yfinance as yf
import bs4 as bs
import pickle
import requests
import numpy as np


Web-scrapping API "BeautifulSoup" is used to get the tickers from Wikipedia:

In [2]:
# function to get the tickers using webscrapping from wikipedia
def sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)
    
    tickers=[s[:-1] for s in tickers] #adjust strings
    
    with open("sp500tickers.pickle","wb") as f: #list of tickers is stored so we don't have to run this everytime
        pickle.dump(tickers,f)
        
    return tickers

tickers=sp500_tickers()

We use Yahoo Finance API to get the data: 

In [7]:
#function to download data from Yahoo Finance 
def get_data(tickers,date_start,date_end):
    df = pd.DataFrame(columns=tickers)
    for ticker in tickers:
        df[ticker]=yf.download(ticker,date_start,date_end)['Adj Close']
        #here we could also implement a pickle object to avoid further download
    return df

# As input we take the ticker list and the start and end date
df=get_data(tickers,"2010-01-01","2019-01-01")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- NLOK: Data doesn't exist for startDate = 1262300400, endDate = 1546297200
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[******

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Data has to be cleaned a bit because a few companies seem to be delisted already

In [8]:
df_clean=df.dropna(axis=1)
tickers_clean=df_clean.columns

Our code is going to consist of two classes: 
    1. Returns --> will output annualized return and annualized standard deviation as well as standard deviation for negative returns (needed for Sortino)
    2. RaR --> will output the risk adjust ratios Sharpe and Sortino for the given portfolio. Also we calculate the weights with respecet to the RaR (tbd)
    
Please note that this is still work in progress!

In [11]:
class Returns:
    def __init__(self,data,tickers): #as an input the data and a list of tickers is needed
        self.data=data
        self.tickers=tickers
    #compute annualized returns    
    def annualized_return(self,year):
        total_return = pd.DataFrame(columns=self.tickers)
        for ticker in self.data:
            total_return[ticker]=pd.Series((self.data[ticker][-1]-self.data[ticker][0])/self.data[ticker][0])
        annualized_return_raw=total_return.apply(lambda num: (((1 + num)**(1/year))-1))
        annualized_return_melt=annualized_return_raw.melt()["value"]
        annualized_return_melt.index=self.tickers
        return annualized_return_melt
    #compute standard deviation of the stock series
    def std_df(self):
        std_df=self.data.pct_change().std()*np.sqrt(250)
        return std_df
    #compute standard deviation of the negative returns
    def std_df_neg(self):
        df_clean_returns=self.data.pct_change()
        std_df_neg=df_clean_returns[df_clean_returns < 0].std()*np.sqrt(250)     
        return std_df_neg
    

returns=Returns(df_clean,tickers_clean) #initialize object
annualized_returns=returns.annualized_return(9) #compute annualized return for a given time period
std_df=returns.std_df() #compute std. deviation for each stock
std_df_neg=returns.std_df_neg() #compute std. devation for each stock wrt. to negative returns only!

In the following class we compute the Sharpe Ratio and the Sortino-Ratio. As an input it takes a risk free rate, the computed annualized returns and std-devations of the Returns-object. As an output we get the Risk-Adjusted-Ratios for each stock for the given input. It also can provide you portfolio-weights from which indices could be build.

In [14]:
class RAR: 
    def __init__(self,rfr,annualized_return):#as an input a risk_free_rate as well as annualized returns are needed
        self.rfr=rfr
        self.annualized_return=annualized_return

    def risk_ratio(self,std_df): #compute risk adjusted ratio
        risk_ratio=(self.annualized_return-self.rfr)/std_df
        return risk_ratio.sort_values()
    
    def risk_ratio_weights(self,std_df): #this should provide you with risk-adjusted weights from which a portfolio/index can be build
        risk_ratio=(self.annualized_return-self.rfr)/std_df
        risk_ratio_pos=risk_ratio[risk_ratio>0] #negative ratios are note taken into account
        risk_ratio_weights=risk_ratio_pos/risk_ratio_pos.sum()
        return risk_ratio_weights.sort_values()
    
    
rfr=0.01
RAR=RAR(rfr,annualized_returns)
sharpe_ratio=RAR.risk_ratio(std_df)#for sharpe we take the standard deviation of positive and negative returns of each stock
sortino_ratio=RAR.risk_ratio(std_df_neg)# for sortino we take the standard deviation of only negative returns of each stock
print(sharpe_ratio)
print(sortino_ratio)
sharpe_ratio_weights=RAR.risk_ratio_weights(std_df)
sortino_ratio_weights=RAR.risk_ratio_weights(std_df_neg)
print(sharpe_ratio_weights)
print(sortino_ratio_weights)


APA    -0.404462
DVN    -0.336875
FCX    -0.256887
ARNC   -0.226400
GE     -0.209121
          ...   
HD      1.194032
UNH     1.211593
EXR     1.233856
TDG     1.281542
NI      1.311717
Length: 452, dtype: float64
APA    -0.600727
DVN    -0.494947
FCX    -0.370553
ARNC   -0.309196
SLB    -0.294652
          ...   
HD      1.727471
MKTX    1.733992
EXR     1.774186
UNH     1.777863
NI      1.811911
Length: 452, dtype: float64
GS      0.000033
XRAY    0.000047
IBM     0.000084
TPR     0.000127
WU      0.000130
          ...   
HD      0.005108
UNH     0.005183
EXR     0.005278
TDG     0.005482
NI      0.005611
Length: 427, dtype: float64
GS      0.000032
XRAY    0.000042
IBM     0.000078
WU      0.000108
TPR     0.000115
          ...   
HD      0.005288
MKTX    0.005308
EXR     0.005431
UNH     0.005443
NI      0.005547
Length: 427, dtype: float64
